In [1]:
# We import the libraries
import os, shutil
import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import cv2
from Utils import *

In [2]:
# We initiate our environment variables
# Directory where you'll store your dataset
base_dir = '../src/food-101/'
categories = ['sashimi', 'spaghetti_bol']
dataset_steps = ['train', 'validation', 'test']

In [3]:
# We call the classes
do = DataOrganizer(base_dir=base_dir,
                  category_list=categories,
                  type_dir_list=dataset_steps)

# Part I: Splitting the Data

In [4]:
# Directories for the training, validation, and test splits
train_dir, validation_dir, test_dir = do.creating_step_directories()
type_dirs = [train_dir, validation_dir, test_dir]
do.creating_dataset_directories()

sashimi_names_list, spaghetti_bol_list = do.selecting_data_names()
names_lists = [sashimi_names_list, spaghetti_bol_list]
do.creating_datasets(names_lists=names_lists)

# Part II: Preparing the Data

In [5]:
dp = DataPreparator(rescale=1./255,
                    type_dir_list=type_dirs,
                    target_size=(150,150))

In [6]:
train_datagen, validation_datagen, test_datagen = dp.image_rescaler()
data_generators = [train_datagen, validation_datagen, test_datagen]

In [7]:
train_generator, validation_generator, test_generator = dp.type_generator(data_generators=data_generators)

Found 1400 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


# Part III: Building the Models

In [ ]:
# Features extractor
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                       input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

In [ ]:
# Classifier
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [ ]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=70,
                epochs=30,
                validation_data=validation_generator,
                validation_steps=15)

In [ ]:
model.save('../models/conv_net_20201228_1.h5')

In [ ]:
new_model = keras.models.load_model('../models/conv_net_20201228_1.h5')

In [ ]:
history.history

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
sashimi_sample = sashimi_train_names[0]
spaghetti_bol_sample = spaghetti_bol_train_names[0]
img_sashimi = image.load_img(sashimi_sample, target_size=(150, 150))
print(type(img_sashimi))

In [ ]:
img_sashimi

In [ ]:
x_sashimi = image.img_to_array(img_sashimi)
x_sashimi = x_sashimi.reshape((1,) + x_sashimi.shape)

In [ ]:
i = 0
for batch in datagen.flow(x_sashimi, batch_size=1):
    plt.figure()
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i = i + 1
    if i % 4 == 0:
        break
plt.show()

In [ ]:
# Features extractor
model_drop = models.Sequential()
model_drop.add(layers.Conv2D(32, (3, 3), activation='relu',
                       input_shape=(150, 150, 3)))
model_drop.add(layers.MaxPooling2D((2, 2)))
model_drop.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_drop.add(layers.MaxPooling2D((2, 2)))
model_drop.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_drop.add(layers.MaxPooling2D((2, 2)))
model_drop.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_drop.add(layers.MaxPooling2D((2, 2)))

In [ ]:
# Classifier with dropout
model_drop.add(layers.Flatten())
model_drop.add(layers.Dropout(0.2))
model_drop.add(layers.Dense(512, activation='relu'))
model_drop.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model_drop.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [ ]:
# training model_drop using data augmentation
train_datagen_drop = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')

test_datagen_drop = ImageDataGenerator(rescale=1./255)

train_generator_drop = train_datagen_drop.flow_from_directory(
                                            train_dir,
                                            target_size=(150, 150),
                                            batch_size=20,
                                            class_mode='binary')

validation_generator_drop = test_datagen_drop.flow_from_directory(
                                            validation_dir,
                                            target_size=(150, 150),
                                            batch_size=20,
                                            class_mode='binary')

In [ ]:
history_drop = model_drop.fit_generator(
                            train_generator_drop,
                            steps_per_epoch=70,
                            epochs=40,
                            validation_data=validation_generator_drop,
                            validation_steps=15)

In [ ]:
nn_performanceperformance(history_drop)

In [ ]:
model_drop.save('../models/conv_net_20201229_1.h5')

In [ ]:
conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))

In [ ]:
conv_base.summary()

In [ ]:
# feature extraction without data augmentation

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [ ]:
train_features, train_labels = extract_features(train_dir, 1400)

In [ ]:
validation_features, validation_labels = extract_features(validation_dir, 300)

In [ ]:
test_features, test_labels = extract_features(test_dir, 300)

In [ ]:
train_features = np.reshape(train_features, (1400, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (300, 4 * 4 * 512))
test_features = np.reshape(test_features, (300, 4 * 4 * 512))

In [ ]:
model_conv = models.Sequential()
model_conv.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model_conv.add(layers.Dropout(0.2))
model_conv.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model_conv.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=2e-5),
             metrics=['acc'])

In [ ]:
history_conv = model_conv.fit(train_features, train_labels,
                             epochs=40,
                             batch_size=20,
                             validation_data=(validation_features, validation_labels))

In [ ]:
nn_performance(history_conv)

In [ ]:
model_conv.save('../models/conv_net_20201229_2.h5')

In [ ]:
# Using pre-trained net with data augmentation

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))

model_conv_aug = models.Sequential()
model_conv_aug.add(conv_base)
model_conv_aug.add(layers.Flatten())
model_conv_aug.add(layers.Dense(256, activation='relu'))
model_conv_aug.add(layers.Dense(1, activation='sigmoid'))

print(len(model_conv_aug.trainable_weights))
conv_base.trainable = False
print(len(model_conv_aug.trainable_weights))


In [ ]:
# training model_drop using data augmentation
train_datagen_conv_aug = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')

test_datagen_conv_aug = ImageDataGenerator(rescale=1./255)

train_generator_conv_aug = train_datagen_conv_aug.flow_from_directory(
                                            train_dir,
                                            target_size=(150, 150),
                                            batch_size=20,
                                            class_mode='binary')

validation_generator_conv_aug = test_datagen_conv_aug.flow_from_directory(
                                            validation_dir,
                                            target_size=(150, 150),
                                            batch_size=20,
                                            class_mode='binary')

In [ ]:
model_conv_aug.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=2e-5),
             metrics=['acc'])

In [ ]:
history_conv_aug = model_conv_aug.fit_generator(
                            train_generator_conv_aug,
                            steps_per_epoch=70,
                            epochs=3,
                            validation_data=validation_generator_conv_aug,
                            validation_steps=15)

In [ ]:
model_conv_aug.save('../models/conv_net_20201230_1.h5')

In [ ]:
model_conv_aug.summary()

In [ ]:
print(len(conv_base.trainable_weights))

In [ ]:
# Fine Tuning
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
conv_base.layers

In [ ]:
# new_model = keras.models.load_model('path_to_my_model.h5')

In [ ]:
model_tuning = keras.models.load_model('../models/conv_net_20201230_1.h5')

In [ ]:
model_tuning.summary()

In [ ]:
model_tuning.compile(loss='binary_crossentropy',
                    optimizer=optimizers.RMSprop(lr=1e-5),
                    metrics=['acc'])

In [ ]:
history_tuning = model_tuning.fit_generator(train_generator_conv_aug,
                                           steps_per_epoch=70,
                                           epochs=3,
                                           validation_data=validation_generator_conv_aug,
                                           validation_steps=15)

In [ ]:
nn_performance(history_tuning)

In [ ]:
model_tuning.save('../models/conv_net_20201230_2.h5')

In [ ]:
nn_performance_smoothed(history_tuning)

In [ ]:
test_generator = test_datagen_conv_aug.flow_from_directory(
                                test_dir,
                                target_size=(150, 150),
                                batch_size=20,
                                class_mode='binary')

test_loss, test_acc = model_tuning.evaluate_generator(test_generator, steps=15)
print('test_acc', test_acc)

In [ ]:
# visualizing intermediate activations
model_vis = keras.models.load_model('../models/conv_net_20201229_1.h5')

In [ ]:
model_vis.summary()

In [ ]:
img_path = '../src/food-101/test/sashimi/1039599.jpg'

img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
print(img_tensor.shape)

img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255. # the model was trained on inputs that were preprocessed this way
print(img_tensor.shape)

In [ ]:
plt.imshow(img_tensor[0])
plt.show()

In [ ]:
layer_outputs = [layer.output for layer in model_vis.layers[:8]]
activation_model = models.Model(inputs=model_vis.input, outputs=layer_outputs)

In [ ]:
activations = activation_model.predict(img_tensor)
print(len(activations))

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
for i in activations:
    plt.matshow(i[0, :, :, 8], cmap='viridis')

In [ ]:
# visualizing every channel in every intermediate activation
layer_names = []
for layer in model_vis.layers[:8]:
    layer_names.append(layer.name)

images_per_row = 16

for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[-1]
    
    size = layer_activation.shape[1] # the feature map has shape (l, size, size, n_features)
    
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))
        
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row]
        
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, 
                         row * size : (row + 1) * size] = channel_image
        
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
# visualizing convnet filters
# defining the loss tensor for filter visualization

model_filters = VGG16(weights='imagenet',
                     include_top=False)

layer_name = 'block3_conv1'
filter_index = 0

layer_output = model_filters.get_layer(layer_name).output
loss = K.mean(layer_output[:, :, :, filter_index])

In [ ]:
# obtaining the gradient of the loss with regard to the input
tf.compat.v1.disable_eager_execution()
grads = K.gradients(loss, model_filters.input)[0]

In [ ]:
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

In [ ]:
# Fetching Numpy output values given Numpy input values
iterate = K.function([model_filters.input], [loss, grads])

loss_value, grads_value = iterate([np.zeros((1, 150, 150, 3))])

In [ ]:
# loss maximation via stochastic gradient descent
input_img_data = np.random.random((1, 150, 150, 3)) * 20 + 128

step = 1.
for i in range(40):
    loss_value, grads_value = iterate([input_img_data])
    input_img_data += grads_value * step

In [ ]:
plt.imshow(generate_pattern('block3_conv1', 0))

In [ ]:
# generating a grid of all filter response patterns in a layer
layer_name = 'block1_conv1'
size = 64
margin = 5

results = np.zeros((8 * size + 7 * margin, 8 * size + 7 * margin, 3))

for i in range(8):
    for j in range(8):
        filter_img = generate_pattern(layer_name, i + (j * 8), size=size)
        
        horizontal_start = i * size + i * margin
        horizontal_end = horizontal_start + size
        vertical_start = j * size + j * margin
        vertical_end = vertical_start + size
        results[horizontal_start: horizontal_end,
                vertical_start: vertical_end, :] = filter_img
results = results.astype(int)
        
plt.figure(figsize=(20, 20))
plt.imshow(results)

In [ ]:
# Visualizing heatmaps of class activation
# cam visualization
model_cam = VGG16(weights='imagenet')

In [ ]:
img_path = '../src/food-101/test/spaghetti_bol/1915989.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
preds = model_cam.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
np.argmax(preds[0])

In [ ]:
spaghetti_output = model_cam.output[:, 959]
last_conv_layer = model_cam.get_layer('block5_conv3')

grads = K.gradients(spaghetti_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads, axis=(0, 1, 2))

iterate = K.function([model_cam.input],
                    [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

heatmap = np.mean(conv_layer_output_value, axis=-1)

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.figure(figsize=(50, 50))
plt.matshow(heatmap)

In [ ]:
img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img
cv2.imwrite('../src/food-101/test/superimposed.jpg', superimposed_img)

In [ ]:
img_path